In [ ]:
# VGG16 on IDRiD

In [ ]:
# Importing library modules

from google.colab import files
import numpy as np
import matplotlib.pyplot as plt
import glob
import cv2
from google.colab.patches import cv2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from keras.models import Model, Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
import os
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import pandas as pd
import numpy as np
import cv2

IMG_SIZE = 128

def load_idrid_data(image_dir, label_csv):
    df = pd.read_csv(label_csv)
    df.columns = df.columns.str.strip()
    X, y = [], []

    for _, row in df.iterrows():
        img_file = row['Image name'].strip() + ".jpg"
        label = int(row['Retinopathy grade'])
        path = os.path.join(image_dir, img_file)

        img = cv2.imread(path)
        if img is not None:
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            X.append(img / 255.0)
            y.append(label)

    return np.array(X), np.array(y)

# Paths
base_path = "/content/drive/MyDrive/Grading_IDRiD"

train_dir = os.path.join(base_path, "1. Original Images", "a. Training Set")
test_dir  = os.path.join(base_path, "1. Original Images", "b. Testing Set")

train_csv = os.path.join(base_path, "2. Groundtruths", "a. IDRiD_Disease Grading_Training Labels.csv")
test_csv  = os.path.join(base_path, "2. Groundtruths", "b. IDRiD_Disease Grading_Testing Labels.csv")

# Load data
x_train, y_train = load_idrid_data(train_dir, train_csv)
x_test, y_test   = load_idrid_data(test_dir, test_csv)

print("Train:", x_train.shape, "Test:", x_test.shape)

Train: (413, 128, 128, 3) Test: (103, 128, 128, 3)


In [ ]:
from sklearn import preprocessing
from tensorflow.keras.utils import to_categorical

# Encode integer class labels
le = preprocessing.LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

# One-hot encode for softmax output
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [ ]:
from tensorflow.keras.applications import VGG16

model = VGG16(input_shape=(128, 128, 3), weights='imagenet', include_top=False)
print(len(model.layers))

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
19


In [ ]:
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model

activation = 'sigmoid'
x = model.output
x = Flatten()(x)
x = Dense(128, activation=activation, kernel_initializer='he_uniform')(x)
prediction_layer = Dense(5, activation='softmax')(x)

cnn_model = Model(inputs=model.input, outputs=prediction_layer)


In [ ]:
for layers in cnn_model.layers[:19]:
    print(layers)
    layers.trainable = False

cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
print(cnn_model.summary())

<InputLayer name=input_layer, built=True>
<Conv2D name=block1_conv1, built=True>
<Conv2D name=block1_conv2, built=True>
<MaxPooling2D name=block1_pool, built=True>
<Conv2D name=block2_conv1, built=True>
<Conv2D name=block2_conv2, built=True>
<MaxPooling2D name=block2_pool, built=True>
<Conv2D name=block3_conv1, built=True>
<Conv2D name=block3_conv2, built=True>
<Conv2D name=block3_conv3, built=True>
<MaxPooling2D name=block3_pool, built=True>
<Conv2D name=block4_conv1, built=True>
<Conv2D name=block4_conv2, built=True>
<Conv2D name=block4_conv3, built=True>
<MaxPooling2D name=block4_pool, built=True>
<Conv2D name=block5_conv1, built=True>
<Conv2D name=block5_conv2, built=True>
<Conv2D name=block5_conv3, built=True>
<MaxPooling2D name=block5_pool, built=True>


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 128, 128, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 128, 128, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 64, 64, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 64, 64, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 64, 64, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 32, 32, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 32, 32, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 32, 32, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 32, 32, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 16, 16, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 16, 16, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 16, 16, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 16, 16, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 8, 8, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 8, 8, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 8, 8, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 8, 8, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 4, 4, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 8192)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │     1,048,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 5)              │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,764,037 (60.14 MB)

 Trainable params: 1,049,349 (4.00 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

None


In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint = ModelCheckpoint("cnn_model.keras", monitor='accuracy', verbose=1, save_best_only=True)
early = EarlyStopping(monitor='accuracy', patience=5, verbose=1)

history = cnn_model.fit(
    x_train, y_train,
    epochs=20,
    callbacks=[checkpoint, early]
)

Epoch 1/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 0.3269 - loss: 0.5310
Epoch 1: accuracy improved from -inf to 0.37530, saving model to cnn_model.keras
13/13 ━━━━━━━━━━━━━━━━━━━━ 94s 7s/step - accuracy: 0.3304 - loss: 0.5283
Epoch 2/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.3559 - loss: 0.4557
Epoch 2: accuracy improved from 0.37530 to 0.38257, saving model to cnn_model.keras
13/13 ━━━━━━━━━━━━━━━━━━━━ 136s 7s/step - accuracy: 0.3578 - loss: 0.4560
Epoch 3/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 0.4360 - loss: 0.4430
Epoch 3: accuracy improved from 0.38257 to 0.45521, saving model to cnn_model.keras
13/13 ━━━━━━━━━━━━━━━━━━━━ 144s 7s/step - accuracy: 0.4373 - loss: 0.4424
Epoch 4/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.4277 - loss: 0.4238
Epoch 4: accuracy improved from 0.45521 to 0.46247, saving model to cnn_model.keras
13/13 ━━━━━━━━━━━━━━━━━━━━ 139s 7s/step - accuracy: 0.4302 - loss: 0.4235
Epoch 5/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s

In [ ]:
model.save('VGG_model.keras')